## Importing Packags

In [51]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

## Appliances Energy usage data set
* How much energy is used by appliances 
* How much energy is used by lights
* T_out is output temperature

In [52]:
df=pd.read_csv('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/appliance_energy_usage.csv', index_col=0, parse_dates=True)

In [53]:
df

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
date,,,,,,,,
2016-01-11 17:00:00,60,30,6.600000,733.5,92.000000,7.000000,5.300000,63.000000
2016-01-11 17:10:00,60,30,6.483333,733.6,92.000000,6.666667,5.200000,59.166667
2016-01-11 17:20:00,50,30,6.366667,733.7,92.000000,6.333333,5.100000,55.333333
2016-01-11 17:30:00,50,40,6.250000,733.8,92.000000,6.000000,5.000000,51.500000
2016-01-11 17:40:00,60,40,6.133333,733.9,92.000000,5.666667,4.900000,47.666667
...,...,...,...,...,...,...,...,...
2016-05-27 17:20:00,100,0,22.733333,755.2,55.666667,3.333333,13.333333,23.666667
2016-05-27 17:30:00,90,0,22.600000,755.2,56.000000,3.500000,13.300000,24.500000
2016-05-27 17:40:00,270,10,22.466667,755.2,56.333333,3.666667,13.266667,25.333333


Data is recorded in every 10 minutes

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19735 entries, 2016-01-11 17:00:00 to 2016-05-27 18:00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Appliances   19735 non-null  int64  
 1   lights       19735 non-null  int64  
 2   T_out        19735 non-null  float64
 3   Press_mm_hg  19735 non-null  float64
 4   RH_out       19735 non-null  float64
 5   Windspeed    19735 non-null  float64
 6   Tdewpoint    19735 non-null  float64
 7   Visibility   19735 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 1.4 MB


In [55]:
df=df.resample('1H').mean()

The above code takes original time series data and converts it into a new DataFrame where the data is aggregated and averaged for each hour. This can be useful for various purposes, such as reducing the granularity of the data or preparing it for further analysis at a lower frequency.

In [56]:
df

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
date,,,,,,,,
2016-01-11 17:00:00,55.000000,35.000000,6.308333,733.750000,92.000000,6.166667,5.050000,53.416667
2016-01-11 18:00:00,176.666667,51.666667,5.941667,734.266667,91.583333,5.416667,4.658333,40.000000
2016-01-11 19:00:00,173.333333,25.000000,6.000000,734.791667,89.750000,6.000000,4.391667,40.000000
2016-01-11 20:00:00,125.000000,35.000000,6.000000,735.283333,87.583333,6.000000,4.016667,40.000000
2016-01-11 21:00:00,103.333333,23.333333,5.833333,735.566667,87.416667,6.000000,3.816667,40.000000
...,...,...,...,...,...,...,...,...
2016-05-27 14:00:00,101.666667,1.666667,21.916667,755.800000,59.000000,2.000000,13.475000,21.583333
2016-05-27 15:00:00,76.666667,0.000000,22.216667,755.675000,57.333333,2.000000,13.258333,21.833333
2016-05-27 16:00:00,135.000000,0.000000,22.883333,755.375000,55.000000,2.416667,13.283333,22.583333


In [57]:
df.corr()

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
Appliances,1.000000,0.261794,0.127453,-0.044034,-0.194515,0.112477,0.021073,-0.003380
lights,0.261794,1.000000,-0.085036,-0.012278,0.078925,0.069831,-0.041437,0.023114
T_out,0.127453,-0.085036,1.000000,-0.143233,-0.574353,0.193064,0.792621,-0.081857
Press_mm_hg,-0.044034,-0.012278,-0.143233,1.000000,-0.092555,-0.236864,-0.244271,0.042413
RH_out,-0.194515,0.078925,-0.574353,-0.092555,1.000000,-0.175334,0.033267,0.087726
Windspeed,0.112477,0.069831,0.193064,-0.236864,-0.175334,1.000000,0.127627,-0.007935
Tdewpoint,0.021073,-0.041437,0.792621,-0.244271,0.033267,0.127627,1.000000,-0.045001
Visibility,-0.003380,0.023114,-0.081857,0.042413,0.087726,-0.007935,-0.045001,1.000000


In [58]:
# Making the list of colors
color_list = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]


def Visualize(data):
    #  This line extracts the names of numerical columns from the DataFrame df and stores them in the features list. 
    # It uses the .select_dtypes() method to filter columns that have a data type of a number (e.g., integers or floats).
    features= list(df.select_dtypes(include=[np.number]).columns.values)
    feature_size=len(features)
    fig, axes = plt.subplots(
        nrows=int(np.ceil(feature_size/2)), ncols=2, figsize=(14, feature_size*2), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(feature_size):
        key = features[i]
        c = color_list[i % (len(color_list))]
        t_data = data[key]
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{}".format(key),
            rot=25,
        )
        ax.legend([key])
    plt.tight_layout()


Visualize(df)

## Testing for augmented dickey fuller test 

Null Hypothesis - Series possesses a unit root and hence is not stationary

Alternate Hypothesis - Series is stationary

In [ ]:
for i in range(len(df.columns)):
  result = adfuller(df[df.columns[i]])

  if result[1] > 0.05 :
    print('{} - Series is not Stationary'.format(df.columns[i]))
  else:
    print('{} - Series is Stationary'.format(df.columns[i]))

Appliances - Series is Stationary
lights - Series is Stationary
T_out - Series is Stationary
Press_mm_hg - Series is Stationary
RH_out - Series is Stationary
Windspeed - Series is Stationary
Tdewpoint - Series is Stationary
Visibility - Series is Stationary


## Granger Causes Test

The Granger Causality Test is a statistical hypothesis test used to determine whether one time series can predict another time series. It is commonly employed in the field of econometrics and time series analysis to assess the causal relationship between two variables, particularly in the context of forecasting.

The Granger Causality Test is based on the idea that if variable X "Granger causes" variable Y, then past values of X should contain information that helps predict the current and future values of Y better than just using past values of Y alone.

H0: Xt does not granger causes Yt

H1: Xt granger causes Yt

In [ ]:
max_lags=8
y='Appliances'

In [ ]:
for i in range(len(df.columns)-1):
  results=grangercausalitytests(df[[y,df.columns[i+1]]], max_lags, verbose=False)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('Column - {} : P_Values - {}'.format(df.columns[i+1],p_values))



# Granger causality is a statistical test used to determine if one time series can predict anothe

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 1
  warnings.warn('covariance of co

Column - lights : P_Values - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Column - T_out : P_Values - [0.3095, 0.0769, 0.1612, 0.0935, 0.0461, 0.0, 0.0, 0.0]
Column - Press_mm_hg : P_Values - [0.4625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


Column - RH_out : P_Values - [0.9365, 0.0207, 0.0383, 0.0387, 0.0057, 0.0, 0.0, 0.0]
Column - Windspeed : P_Values - [0.0095, 0.0019, 0.0004, 0.0002, 0.0001, 0.0001, 0.0001, 0.0]


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


Column - Tdewpoint : P_Values - [0.2621, 0.0865, 0.1409, 0.2077, 0.1799, 0.2621, 0.3403, 0.3027]
Column - Visibility : P_Values - [0.9945, 0.8036, 0.2634, 0.4584, 0.4403, 0.3354, 0.3802, 0.2764]


In [ ]:
max_lags=8
y='lights'
for i in range(len(df.columns)-1):
  results=grangercausalitytests(df[[y,df.columns[i+1]]], max_lags)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('Column - {} : P_Values - {}'.format(df.columns[i+1],p_values))


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=-43.4078, p=1.0000  , df_denom=3286, df_num=1
ssr based chi2 test:   chi2=-43.4475, p=1.0000  , df=1
likelihood ratio test: chi2=-43.7370, p=1.0000  , df=1
parameter F test:         F=28.1999 , p=0.0000  , df_denom=3286, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=-1.7785 , p=1.0000  , df_denom=3284, df_num=2
ssr based chi2 test:   chi2=-3.5614 , p=1.0000  , df=2
likelihood ratio test: chi2=-3.5633 , p=1.0000  , df=2
parameter F test:         F=7.6488  , p=0.0057  , df_denom=3284, df_num=1

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=-2.7467 , p=1.0000  , df_denom=3282, df_num=3
ssr based chi2 test:   chi2=-8.2527 , p=1.0000  , df=3
likelihood ratio test: chi2=-8.2631 , p=1.0000  , df=3
parameter F test:         F=14.5413 , p=0.0001  , df_denom=3282, df_num=1

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=-0.7767 , p=1.

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 1
  warnings.warn('covariance of constraints does not have full '
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. Th

ssr based F test:         F=8.0390  , p=0.0000  , df_denom=3265, df_num=8
ssr based chi2 test:   chi2=64.6467 , p=0.0000  , df=8
likelihood ratio test: chi2=64.0183 , p=0.0000  , df=8
parameter F test:         F=8.0390  , p=0.0000  , df_denom=3265, df_num=8
Column - T_out : P_Values - [0.3095, 0.0769, 0.1612, 0.0935, 0.0461, 0.0, 0.0, 0.0]

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.5399  , p=0.4625  , df_denom=3286, df_num=1
ssr based chi2 test:   chi2=0.5404  , p=0.4623  , df=1
likelihood ratio test: chi2=0.5403  , p=0.4623  , df=1
parameter F test:         F=0.5399  , p=0.4625  , df_denom=3286, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.2075 , p=0.0000  , df_denom=3283, df_num=2
ssr based chi2 test:   chi2=24.4523 , p=0.0000  , df=2
likelihood ratio test: chi2=24.3618 , p=0.0000  , df=2
parameter F test:         F=12.2075 , p=0.0000  , df_denom=3283, df_num=2

Granger Causality
number of lags (no zero) 3
ssr 

In [ ]:
df_input=df[['Appliances','T_out','Windspeed']]
df_input

,Appliances,T_out,Windspeed
date,,,
2016-01-11 17:00:00,55.000000,6.308333,6.166667
2016-01-11 18:00:00,176.666667,5.941667,5.416667
2016-01-11 19:00:00,173.333333,6.000000,6.000000
2016-01-11 20:00:00,125.000000,6.000000,6.000000
2016-01-11 21:00:00,103.333333,5.833333,6.000000
...,...,...,...
2016-05-27 14:00:00,101.666667,21.916667,2.000000
2016-05-27 15:00:00,76.666667,22.216667,2.000000
2016-05-27 16:00:00,135.000000,22.883333,2.416667


In [ ]:
df_train = df_input[:int(0.9*(len(df_input)))]
df_test = df_input[int(0.9*(len(df_input))):]

In [ ]:
df_train.shape

(2961, 3)

In [ ]:
df_test.shape

(329, 3)

## var model fitting


In [ ]:
model = VAR(df_train, freq="1H")
for i in range(48):
    results = model.fit(i+1)
    print('Order = ', i+1)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)

Order =  1
AIC:  7.126785098024288
BIC:  7.151080819162064
Order =  2
AIC:  6.13876916892782
BIC:  6.181298651759994
Order =  3
AIC:  6.019982255185912
BIC:  6.0807557708377455
Order =  4
AIC:  5.9605267907597215
BIC:  6.039554619733089
Order =  5
AIC:  5.908069667837072
BIC:  6.00536210002221
Order =  6
AIC:  5.891197292905782
BIC:  6.006764627593054
Order =  7
AIC:  5.85225035501184
BIC:  5.986102900903522


Order =  8
AIC:  5.841915871050664
BIC:  5.994063946272741
Order =  9
AIC:  5.827889041110309
BIC:  5.998342973224303
Order =  10
AIC:  5.825120732509795
BIC:  6.013890858524602
Order =  11
AIC:  5.791083147024334
BIC:  5.998179813408081
Order =  12
AIC:  5.78322849534456
BIC:  6.008662058036488
Order =  13
AIC:  5.764122425516564
BIC:  6.00790324993892
Order =  14
AIC:  5.7545168062341
BIC:  6.016655267304056
Order =  15
AIC:  5.745057263179905
BIC:  6.025563745321492
Order =  16
AIC:  5.734159567395153
BIC:  6.033044464551212
Order =  17
AIC:  5.734437324119505
BIC:  6.051711039763665
Order =  18
AIC:  5.727535892320577
BIC:  6.063208839469243
Order =  19
AIC:  5.7180404229514785
BIC:  6.072123024175843
Order =  20
AIC:  5.7195359540939625
BIC:  6.092038641532039
Order =  21
AIC:  5.722525512492248
BIC:  6.113458727860916
Order =  22
AIC:  5.719011568286694
BIC:  6.128385762893773
Order =  23
AIC:  5.683257270961978
BIC:  6.111082905718311
Order =  24
AIC:  5.669514653667391
BIC:  6.

In [ ]:
model.select_order(48).summary()

,AIC,BIC,FPE,HQIC
0,13.82,13.83,1.005e+06,13.82
1,7.139,7.163,1260.,7.148
2,6.149,6.192,468.3,6.165
3,6.031,6.093,416.2,6.053
4,5.970,6.050,391.7,5.999
5,5.917,6.015,371.3,5.952
6,5.902,6.019,365.7,5.944
7,5.863,5.999*,351.9,5.912
8,5.852,6.006,348.1,5.908
9,5.837,6.010,342.9,5.900


In [ ]:
model = VAR(df_train, freq="1H")
results = model.fit(7)

In [ ]:
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 17, Sep, 2023
Time:                     09:55:03
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    5.98610
Nobs:                     2954.00    HQIC:                   5.90043
Log likelihood:          -21152.4    FPE:                    348.017
AIC:                      5.85225    Det(Omega_mle):         340.356
--------------------------------------------------------------------
Results for equation Appliances
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const                37.865935         3.448219           10.981           0.000
L1.Appliances         0.451436         0.018470           24.442           0.000
L1.T_out              2.328238         2.534185        

In [ ]:
lag=results.k_ar

In [ ]:
results.forecast(df_train.values[-lag:],steps=5)

array([[44.34598018,  8.92651619,  3.99333186],
       [33.70923187,  8.81719814,  4.13871016],
       [38.32344216,  9.01953267,  4.188856  ],
       [53.39756915,  9.3356185 ,  4.31429082],
       [60.93798924,  9.56974211,  4.40093836]])

In [ ]:
df_test[0:5]

,Appliances,T_out,Windspeed
date,,,
2016-05-14 02:00:00,60.000000,8.891667,3.166667
2016-05-14 03:00:00,60.000000,8.725000,2.833333
2016-05-14 04:00:00,60.000000,8.775000,4.416667
2016-05-14 05:00:00,56.666667,8.683333,5.000000
2016-05-14 06:00:00,63.333333,8.716667,5.000000


In [ ]:
df_coeff=pd.DataFrame([results.params['Appliances'],results.pvalues['Appliances']]).T

In [ ]:
df_coeff

,Appliances,Appliances
const,37.865935,4.701032e-28
L1.Appliances,0.451436,6.138160e-132
L1.T_out,2.328238,3.582355e-01
L1.Windspeed,2.268735,2.564126e-01
L2.Appliances,0.222279,4.255512e-28
L2.T_out,-4.372626,4.302427e-01
L2.Windspeed,-1.481847,6.603327e-01
L3.Appliances,-0.019489,3.454670e-01
L3.T_out,5.606923,3.952787e-01
L3.Windspeed,0.184615,9.598961e-01


In [ ]:
df_coeff.columns = ['coeff','pval']

In [ ]:
df_coeff.query('pval < 0.05')
np.where(df_coeff['pval'] < 0.05,1, 0)

array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [ ]:
df_coeff['valid']=np.where(df_coeff['pval'] < 0.05,1, 0)

In [ ]:
df_coeff

,coeff,pval,valid
const,37.865935,4.701032e-28,1
L1.Appliances,0.451436,6.138160e-132,1
L1.T_out,2.328238,3.582355e-01,0
L1.Windspeed,2.268735,2.564126e-01,0
L2.Appliances,0.222279,4.255512e-28,1
L2.T_out,-4.372626,4.302427e-01,0
L2.Windspeed,-1.481847,6.603327e-01,0
L3.Appliances,-0.019489,3.454670e-01,0
L3.T_out,5.606923,3.952787e-01,0
L3.Windspeed,0.184615,9.598961e-01,0


In [ ]:
#coeff_arr=np.multiply(df_coeff['coeff'], df_coeff['valid'])[1:].values
coeff_arr=df_coeff['coeff'][1:].values

In [ ]:
coeff_arr.shape

(21,)

In [ ]:
coeff_arr

array([ 0.45143647,  2.32823846,  2.26873466,  0.2222793 , -4.37262576,
       -1.48184722, -0.01948858,  5.60692324,  0.18461514, -0.04369054,
       -5.43530357, -0.08685622,  0.00738616,  5.17191268, -1.99030973,
       -0.01683822,  3.22703425,  4.97164661, -0.01091878, -6.77733848,
       -2.97259279])

In [ ]:
df_train[-lag:]

,Appliances,T_out,Windspeed
date,,,
2016-05-13 19:00:00,208.333333,19.333333,9.000000
2016-05-13 20:00:00,120.000000,17.066667,8.166667
2016-05-13 21:00:00,63.333333,14.116667,6.583333
2016-05-13 22:00:00,55.000000,11.850000,6.833333
2016-05-13 23:00:00,61.666667,10.466667,7.583333
2016-05-14 00:00:00,58.333333,9.791667,5.750000
2016-05-14 01:00:00,63.333333,9.333333,4.000000


In [ ]:
in_arr=df_train[-lag:][::-1].stack().to_frame().T.values

In [ ]:
in_arr.shape

(1, 21)

In [ ]:
in_arr

array([[ 63.33333333,   9.33333333,   4.        ,  58.33333333,
          9.79166667,   5.75      ,  61.66666667,  10.46666667,
          7.58333333,  55.        ,  11.85      ,   6.83333333,
         63.33333333,  14.11666667,   6.58333333, 120.        ,
         17.06666667,   8.16666667, 208.33333333,  19.33333333,
          9.        ]])

In [ ]:
np.dot(in_arr, coeff_arr)+df_coeff['coeff'][:1].values

array([44.34598018])